In [1]:
import Short_Algo_pi
import lattice
import kakunou
import qseplot
import time
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import scipy.sparse as spsp
import matplotlib.pyplot as plt
import csv
import os

from scipy.spatial import distance
from scipy.optimize import linprog
from scipy.optimize import minimize
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
#ここを実験によって変える．

Col_list = [10, 20, 30]
rel_path_list = []
    #step.1
for Col in Col_list:

    # Scaling * tが格子の最小距離となる。
    Scaling = 3 / Col

    city_network = lattice.make_lattice(Col)
    K = Col * Col

    # 通勤費用パラメータ
    t = 0.1

    # 距離抵抗パラメータ
    tau = 0.01

    # 総土地供給量
    S_total = K
    S_bar = S_total / K

    Coordinate_Data = \
    np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
               city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
    distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

    theta_N = 5.0
    theta_M = 5.0

    E = 5

    alpha_1 = 0.4
    alpha_2 = 0.4
    beta_1 = 0.4
    beta_2 = 0.4

    M = 0.1 * K
    N = 0.1 * K

    L = 0.2
    eta = 1.2
    p_proj = 0.1

    #パラメータ設定
    prm = Short_Algo_pi.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
    algprm = Short_Algo_pi.Algo_Parameter(L, eta, p_proj)
    short = Short_Algo_pi.Short(prm, algprm)

    #長期の変数の設定
    # dF_long = pd.read_excel(fr'C:\Users\kimura\OneDrive - 国立大学法人東北大学\numerical\short_dual\log\excelfile\
    # shortlong\FISTA\theta={prm.theta_N}\E={prm.E}\mesh\K={prm.Col}^2\truevalue\jikken_F.xlsx')
    # dH_long = pd.read_excel(fr'C:\Users\kimura\OneDrive - 国立大学法人東北大学\numerical\short_dual\log\excelfile\
    # shortlong\FISTA\theta={prm.theta_N}\E={prm.E}\mesh\K={prm.Col}^2\truevalue\jikken_H.xlsx')

#     dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
#     fr'shortlong/FISTA/theta={prm.theta_firm}/E=10/mesh/K={prm.Col}^2/truevalue/jikken_F.xlsx')
#     dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
#     fr'shortlong/FISTA/theta={prm.theta_firm}/E=10/mesh/K={prm.Col}^2/truevalue/jikken_H.xlsx')

#     m_fixed = np.array(dF_long['m'].tolist())
#     n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

    m_per = M / K
    m_fixed = np.array([m_per] * K)
    n_fixed = np.full((K, K), N / (K * K))
    
    R = np.ones(K) * 0.1
    W = np.ones(K) * 0.1
    
    pi = np.maximum(0, np.dot(prm.D, m_fixed)\
        - beta_1 * np.log(R) - beta_2 * np.log(W)\
        + beta_1 * np.log(beta_1) + beta_2 * np.log(beta_2)\
        - beta_1 - beta_2)
    v = np.maximum(0, E * W * np.ones((K, K))\
        - (alpha_1 * np.log(R) * np.ones((K, K))).T\
        - alpha_2 * np.log(W) * np.ones((K, K))\
        + alpha_1 * np.log(alpha_1) + alpha_2 * np.log(alpha_2)\
        - alpha_1 - alpha_2 - prm.T)
    
    dF = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'short/FISTA/long_ichiyou/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/K={prm.Col}^2/truevalue/jikken_F.xlsx')
    dH = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'short/FISTA/long_ichiyou/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/K={prm.Col}^2/truevalue/jikken_H.xlsx')
    
    pi_corr = np.array(dF['pi'].tolist())
    pi_err = (pi - pi_corr) / pi_corr
    
    R_corr = np.array(dF['R'].tolist())
    R_err = (R - R_corr) / R_corr
    
    W_corr = np.array(dF['W'].tolist())
    W_err = (W - W_corr) / W_corr
    
    v_corr = np.array(dH['v'].tolist()).reshape(K, K)
    v_err = (v - v_corr) / v_corr
    
    pi_noex = np.maximum(0, 
        - beta_1 * np.log(R) - beta_2 * np.log(W)\
        + beta_1 * np.log(beta_1) + beta_2 * np.log(beta_2)\
        - beta_1 - beta_2)
    
    # pi_noex = - beta_1 * np.log(R) - beta_2 * np.log(W)\
    #     + beta_1 * np.log(beta_1) + beta_2 * np.log(beta_2)\
    #     - beta_1 - beta_2
    
    pi_noex_corr = np.maximum(0, 
        - beta_1 * np.log(R_corr) - beta_2 * np.log(W_corr)\
        + beta_1 * np.log(beta_1) + beta_2 * np.log(beta_2)\
        - beta_1 - beta_2)
    
    Z_SD = np.sum(v * n_fixed)\
    + np.sum(pi_noex * m_fixed)\
    + S_bar * np.sum(R)
    
    Z_SD_corr = np.sum(v_corr * n_fixed)\
    + np.sum(pi_noex_corr * m_fixed)\
    + S_bar * np.sum(R_corr)

    # #step.1-1
    print("Col:", Col)
    
#     matrix = np.reshape(pi, (Col, Col))
#     x, y = np.meshgrid(range(Col), range(Col))
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.plot_wireframe(x, y, matrix, cmap='viridis')
#     ax.set_zlabel('pi_corr')
#     plt.gca().invert_xaxis()
#     plt.show()

#     matrix = np.reshape(pi_corr, (Col, Col))
#     x, y = np.meshgrid(range(Col), range(Col))
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.plot_wireframe(x, y, matrix, cmap='viridis')
#     ax.set_zlabel('pi')
#     plt.gca().invert_xaxis()
#     plt.show()
    
#     print("pi:", pi)
#     print("pi_corr:", pi_corr)
#     print("pi_err:", pi_err)
    
#     print("W:", W)
#     print("W_corr:", W_corr)
#     print("W_err:", W_err)
    
    print("v:", v)
    print("pi_noex:", pi_noex)
    print("v_corr:", v_corr)
    print("pi_noex_corr:", pi_noex_corr)
    print("Z_SD:", Z_SD)
    print("Z_SD_corr:", Z_SD_corr)
    # short.short_solve(m_fixed, n_fixed, short_itr=301, err_short=10**(-8))

Col: 10
v: [[0.80903549 0.77903549 0.74903549 ... 0.46698286 0.44778765 0.42719783]
 [0.77903549 0.80903549 0.77903549 ... 0.48453587 0.46698286 0.44778765]
 [0.74903549 0.77903549 0.80903549 ... 0.50016658 0.48453587 0.46698286]
 ...
 [0.46698286 0.48453587 0.50016658 ... 0.80903549 0.77903549 0.74903549]
 [0.44778765 0.46698286 0.48453587 ... 0.77903549 0.80903549 0.77903549]
 [0.42719783 0.44778765 0.46698286 ... 0.74903549 0.77903549 0.80903549]]
pi_noex: [0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549
 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549 0.30903549


In [7]:
def generate_file_path(filename):
    folder_path = r'C:\Users\kimura\OneDrive - 国立大学法人東北大学'
    file_path = fr'{folder_path}\{filename}.xlsx'
    return file_path

# 使用例
file_name = 'example'
path = generate_file_path(file_name)
print(path)

C:\Users\kimura\OneDrive - 国立大学法人東北大学\example.xlsx
